# MLP Classification with CR Dataset
<hr>

We will build a text classification model using Multi Layer Perceptron on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
from tensorflow.keras.preprocessing.text import Tokenizer
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')


Bad key text.latex.unicode in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
h

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/CR/CR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(3775, 3)


,sentence,label,split
0,weaknesses are minor the feel and layout of th...,0,train
1,many of our disney movies do n 't play on this...,0,train
2,player has a problem with dual layer dvd 's su...,0,train
3,i know the saying is you get what you pay for ...,0,train
4,will never purchase apex again .,0,train
...,...,...,...
3770,"so far , the anti spam feature seems to be ver...",1,train
3771,i downloaded a trial version of computer assoc...,1,train
3772,i did not have any of the installation problem...,1,train
3773,their products have been great and have saved ...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3775 entries, 0 to 3774
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  3775 non-null   object
 1   label     3775 non-null   int32 
 2   split     3775 non-null   object
dtypes: int32(1), object(2)
memory usage: 73.9+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,1368,1368
1,2407,2407


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

"weaknesses are minor the feel and layout of the remote control are only so so . it does n 't show the complete file names of mp3s with really long names . you must cycle through every zoom setting ( 2x , 3x , 4x , 1 2x , etc . ) before getting back to normal size sorry if i 'm just ignorant of a way to get back to 1x quickly ."

<!--## Split Dataset-->

# Data Preprocessing: Word2Vec Static
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

## Load Pre-trained Word Embedding: Word2Vec

__1. Load `Word2Vec` Pre-trained Word Embedding__

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

In [8]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

In [10]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [11]:
oov_tok = '<UNK>'

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 5046 words present from 5336 training vocabulary in the set of pre-trained word vector


## Define `clean_doc` function
__2. Define a function to clean the document called __`clean_doc()`____

In [12]:
def clean_doc(sentences, word_index):
    clean_sentences = []
    for sentence in sentences:
        sentence = sentence.lower().split()
        clean_word = []
        for word in sentence:
            if word in word_index:
                clean_word.append(word)
        clean_sentence = ' '.join(clean_word)
        clean_sentences.append(clean_sentence)
    return clean_sentences

In [13]:
clean_sentences = clean_doc(sentences, word_index)
clean_sentences[0:3]

["weaknesses are minor the feel and layout of the remote control are only so so it does n 't show the complete file names of mp3s with really long names you must cycle through every zoom setting 2x 3x 4x 1 2x etc before getting back to normal size sorry if i 'm just ignorant of a way to get back to 1x quickly",
 "many of our disney movies do n 't play on this dvd player",
 "player has a problem with dual layer dvd 's such as alias season 1 and season 2"]

## Define `sentence_to_avg` function
__3. Define a `sentence_to_avg` function__

We will use this function to calculate the mean of word embedding representation.

In [14]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = (sentence.lower()).split()

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros(word2vec.word_vec('i').shape)
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    total = 0
    count = 0
    for w in words:
        if w in word_to_vec_map:
            total += word_to_vec_map.word_vec(w)
            count += 1
            
    if count!= 0:
        avg = total/count
    else:
        avg
    return avg

In [15]:
i = word2vec.word_vec('i')[0]
print(word2vec.word_vec('i')[0])
j = word2vec.word_vec('am')[0]
print(word2vec.word_vec('am')[0])
k = word2vec.word_vec('handsome')[0]
print(word2vec.word_vec('handsome')[0])
mean = (i+j+k)/3
print('the mean of word embedding is: ', mean)

-0.22558594
-0.16699219
0.11376953
the mean of word embedding is:  -0.09293619791666667


In [16]:
# Example of the functions used in a sentence
mysentence = 'I am handsome'
sentence_to_avg(mysentence, word2vec)

array([-0.0929362 ,  0.03125   , -0.03914388,  0.09879557,  0.07088598,
        0.03092448, -0.00651042, -0.04437256,  0.08068848,  0.07242838,
        0.00160726, -0.10530599, -0.07389323, -0.08854166,  0.00565592,
        0.15136719, -0.0460612 ,  0.19482422,  0.1101888 ,  0.05924479,
       -0.18457031,  0.00716146,  0.16153972,  0.02437337, -0.01578776,
        0.06119792, -0.25048828,  0.02799479,  0.0853475 , -0.14029948,
        0.13688152, -0.01350911, -0.05493164, -0.01090495,  0.03352864,
        0.09635416, -0.04239909,  0.00777181, -0.1438802 ,  0.06510416,
        0.14560954, -0.11295573,  0.25520834,  0.08833822,  0.14339192,
        0.037028  , -0.02832031, -0.00139872,  0.00309245, -0.17871094,
        0.06852213,  0.07910156,  0.09513346,  0.11425781, -0.00488281,
        0.11051432, -0.01139323, -0.08479818, -0.09277344, -0.03263346,
       -0.00374349,  0.07977295, -0.26416016, -0.05135091,  0.06111654,
       -0.06933594, -0.06486002,  0.18766277, -0.04826609,  0.03

## Encode Sentence into Word2Vec Representation

In [17]:
def encoded_sentences(sentences):

    encoded_sentences = []

    for sentence in sentences:

        encoded_sentence = sentence_to_avg(sentence, word2vec)
        encoded_sentences.append(encoded_sentence)

    encoded_sentences = np.array(encoded_sentences)
    return encoded_sentences

In [18]:
embedded_sentences = encoded_sentences(clean_sentences)
print(embedded_sentences.shape)
embedded_sentences

(3775, 300)


array([[ 0.02931269,  0.04064355,  0.00335943, ..., -0.05255696,
        -0.00678974, -0.03428983],
       [ 0.02625621,  0.07036244, -0.00320712, ..., -0.0249717 ,
         0.02348744, -0.04393421],
       [ 0.0471889 ,  0.04338728, -0.02501352, ..., -0.07487269,
         0.00302996, -0.02290562],
       ...,
       [-0.01438395,  0.04383342,  0.0245463 , ..., -0.08561961,
         0.0373319 , -0.09303284],
       [-0.01439794,  0.05486043, -0.01898448, ..., -0.08718872,
         0.02372233, -0.02985636],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

# MLP Model: 1

## Model Definition

In [19]:
def define_model(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=50, activation='relu', input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [20]:
model_0 = define_model(300)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                15050     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 15,101
Trainable params: 15,101
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [21]:
'''
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True
'''
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [22]:
# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Obtain the word to index
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into word embedding average representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model = define_model(Xtrain.shape[1])

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    print()

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Training 1: 
Epoch 1/40
107/107 [==============================] - 1s 8ms/step - loss: 0.6287 - accuracy: 0.6394 - val_loss: 0.5811 - val_accuracy: 0.6640
Epoch 2/40
107/107 [==============================] - 0s 3ms/step - loss: 0.5535 - accuracy: 0.7103 - val_loss: 0.5297 - val_accuracy: 0.7196
Epoch 3/40
107/107 [==============================] - 0s 3ms/step - loss: 0.5081 - accuracy: 0.7580 - val_loss: 0.5057 - val_accuracy: 0.7381
Epoch 4/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4841 - accuracy: 0.7730 - val_loss: 0.4921 - val_accuracy: 0.7407
Epoch 5/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4630 - accuracy: 0.7801 - val_loss: 0.4891 - val_accuracy: 0.7646
Epoch 6/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4553 - accuracy: 0.7851 - val_loss: 0.4761 - val_accuracy: 0.7751
Epoch 7/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4391 - accuracy: 0.7963 - val_loss: 0.4764 - val_accuracy: 

107/107 [==============================] - 0s 3ms/step - loss: 0.3947 - accuracy: 0.8228 - val_loss: 0.4401 - val_accuracy: 0.7937
Epoch 00015: early stopping
Test Accuracy: 79.62962985038757

Training 4: 
Epoch 1/40
107/107 [==============================] - 1s 6ms/step - loss: 0.6356 - accuracy: 0.6294 - val_loss: 0.5909 - val_accuracy: 0.6508
Epoch 2/40
107/107 [==============================] - 0s 3ms/step - loss: 0.5625 - accuracy: 0.7027 - val_loss: 0.5403 - val_accuracy: 0.7143
Epoch 3/40
107/107 [==============================] - 0s 3ms/step - loss: 0.5118 - accuracy: 0.7536 - val_loss: 0.5139 - val_accuracy: 0.7540
Epoch 4/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4796 - accuracy: 0.7739 - val_loss: 0.5024 - val_accuracy: 0.7698
Epoch 5/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4648 - accuracy: 0.7827 - val_loss: 0.4942 - val_accuracy: 0.7725
Epoch 6/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4492 - a

107/107 [==============================] - 0s 4ms/step - loss: 0.4108 - accuracy: 0.8190 - val_loss: 0.4655 - val_accuracy: 0.7692
Epoch 12/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4134 - accuracy: 0.8111 - val_loss: 0.4602 - val_accuracy: 0.7719
Epoch 13/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4027 - accuracy: 0.8152 - val_loss: 0.4690 - val_accuracy: 0.7692
Epoch 14/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4040 - accuracy: 0.8184 - val_loss: 0.4564 - val_accuracy: 0.7745
Epoch 15/40
107/107 [==============================] - 0s 4ms/step - loss: 0.3929 - accuracy: 0.8237 - val_loss: 0.4726 - val_accuracy: 0.7772
Epoch 16/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3955 - accuracy: 0.8214 - val_loss: 0.4563 - val_accuracy: 0.7905
Epoch 17/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3878 - accuracy: 0.8270 - val_loss: 0.4604 - val_accuracy: 0.7825
Epoch 18/40

107/107 [==============================] - 0s 4ms/step - loss: 0.4699 - accuracy: 0.7778 - val_loss: 0.4668 - val_accuracy: 0.7825
Epoch 6/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4533 - accuracy: 0.7928 - val_loss: 0.4574 - val_accuracy: 0.7931
Epoch 7/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4418 - accuracy: 0.7969 - val_loss: 0.4638 - val_accuracy: 0.7586
Epoch 8/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4371 - accuracy: 0.8067 - val_loss: 0.4526 - val_accuracy: 0.7798
Epoch 9/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4270 - accuracy: 0.8072 - val_loss: 0.4480 - val_accuracy: 0.7825
Epoch 10/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4225 - accuracy: 0.8081 - val_loss: 0.4432 - val_accuracy: 0.7825
Epoch 11/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4162 - accuracy: 0.8078 - val_loss: 0.4494 - val_accuracy: 0.7851
Epoch 12/40
107

## Summary

In [23]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,78.306878,79.100531,79.62963,78.306878,83.333331,79.310346,81.43236,79.310346,80.901855,80.901855,80.053401


In [24]:
report = record
report = report.to_excel('Emb_MLP_CR.xlsx', sheet_name='model_1')

<hr>

# MLP Model: 2

## Model Definition

In [25]:
def define_model_2(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [26]:
model_1 = define_model_2(300)
model_1.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 101       
Total params: 30,201
Trainable params: 30,201
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [27]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=1, 
                                             mode='auto', restore_best_weights=True)

In [28]:
# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Obtain the word to index
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into word embedding average representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model = define_model_2(Xtrain.shape[1])

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    print()

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Epoch 1/40
107/107 [==============================] - 1s 6ms/step - loss: 0.6065 - accuracy: 0.6562 - val_loss: 0.5659 - val_accuracy: 0.6534
Epoch 2/40
107/107 [==============================] - 0s 4ms/step - loss: 0.5205 - accuracy: 0.7401 - val_loss: 0.5033 - val_accuracy: 0.7354
Epoch 3/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4808 - accuracy: 0.7680 - val_loss: 0.4740 - val_accuracy: 0.7725
Epoch 4/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4536 - accuracy: 0.7942 - val_loss: 0.4589 - val_accuracy: 0.7566
Epoch 5/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4412 - accuracy: 0.7960 - val_loss: 0.4672 - val_accuracy: 0.7593
Epoch 6/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4295 - accuracy: 0.8007 - val_loss: 0.4493 - val_accuracy: 0.7698
Epoch 7/40
107/107 [==============================] - 0s 4ms/step - loss: 0.4192 - accuracy: 0.8092 - val_loss: 0.4438 - val_accuracy: 0.7646
Epoch 

107/107 [==============================] - 0s 2ms/step - loss: 0.3759 - accuracy: 0.8254 - val_loss: 0.4751 - val_accuracy: 0.7831
Epoch 19/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3673 - accuracy: 0.8322 - val_loss: 0.4771 - val_accuracy: 0.7831
Epoch 20/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3645 - accuracy: 0.8378 - val_loss: 0.4796 - val_accuracy: 0.7751
Epoch 00020: early stopping
Test Accuracy: 78.57142686843872

Training 3: 
Epoch 1/40
107/107 [==============================] - 0s 4ms/step - loss: 0.6148 - accuracy: 0.6582 - val_loss: 0.5585 - val_accuracy: 0.6931
Epoch 2/40
107/107 [==============================] - 0s 2ms/step - loss: 0.5278 - accuracy: 0.7298 - val_loss: 0.5028 - val_accuracy: 0.7513
Epoch 3/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4854 - accuracy: 0.7751 - val_loss: 0.4778 - val_accuracy: 0.7540
Epoch 4/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4580 -

107/107 [==============================] - 0s 2ms/step - loss: 0.4838 - accuracy: 0.7683 - val_loss: 0.4698 - val_accuracy: 0.7884
Epoch 4/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4616 - accuracy: 0.7795 - val_loss: 0.4394 - val_accuracy: 0.8175
Epoch 5/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4520 - accuracy: 0.7919 - val_loss: 0.4328 - val_accuracy: 0.8016
Epoch 6/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4346 - accuracy: 0.8025 - val_loss: 0.4212 - val_accuracy: 0.8254
Epoch 7/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4342 - accuracy: 0.7954 - val_loss: 0.4154 - val_accuracy: 0.8201
Epoch 8/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4173 - accuracy: 0.8134 - val_loss: 0.4126 - val_accuracy: 0.8069
Epoch 9/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4131 - accuracy: 0.8025 - val_loss: 0.4069 - val_accuracy: 0.8228
Epoch 10/40
107/1

107/107 [==============================] - 0s 2ms/step - loss: 0.5313 - accuracy: 0.7216 - val_loss: 0.4941 - val_accuracy: 0.7905
Epoch 3/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4823 - accuracy: 0.7749 - val_loss: 0.4675 - val_accuracy: 0.7984
Epoch 4/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4595 - accuracy: 0.7828 - val_loss: 0.4562 - val_accuracy: 0.8011
Epoch 5/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4445 - accuracy: 0.7958 - val_loss: 0.4431 - val_accuracy: 0.8223
Epoch 6/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4334 - accuracy: 0.7978 - val_loss: 0.4514 - val_accuracy: 0.7851
Epoch 7/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4283 - accuracy: 0.8064 - val_loss: 0.4350 - val_accuracy: 0.8011
Epoch 8/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4148 - accuracy: 0.8099 - val_loss: 0.4435 - val_accuracy: 0.7798
Epoch 9/40
107/10

107/107 [==============================] - 0s 3ms/step - loss: 0.4389 - accuracy: 0.8011 - val_loss: 0.5124 - val_accuracy: 0.7454
Epoch 6/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4256 - accuracy: 0.8034 - val_loss: 0.5034 - val_accuracy: 0.7586
Epoch 7/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4157 - accuracy: 0.8152 - val_loss: 0.4966 - val_accuracy: 0.7613
Epoch 8/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4063 - accuracy: 0.8164 - val_loss: 0.4959 - val_accuracy: 0.7692
Epoch 9/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3999 - accuracy: 0.8199 - val_loss: 0.5012 - val_accuracy: 0.7454
Epoch 10/40
107/107 [==============================] - 0s 2ms/step - loss: 0.3990 - accuracy: 0.8164 - val_loss: 0.4843 - val_accuracy: 0.7905
Epoch 11/40
107/107 [==============================] - 0s 2ms/step - loss: 0.3907 - accuracy: 0.8240 - val_loss: 0.4886 - val_accuracy: 0.7825
Epoch 12/40
107

## Summary

In [29]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,78.306878,78.571427,81.216931,81.216931,82.539684,79.575598,80.106103,82.228118,81.697613,79.045093,80.450438


In [30]:
report = record2
report = report.to_excel('Emb_MLP_CR_2.xlsx', sheet_name='model_2')

<hr>

# MLP Model: 3

## Model Definition

In [31]:
def define_model_3(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=50, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [32]:
model_2 = define_model_3(300)
model_2.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_23 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 51        
Total params: 35,201
Trainable params: 35,201
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [33]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=1, 
                                             mode='auto', restore_best_weights=True)

In [34]:
# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Define the word_index
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into word embedding average representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model = define_model_3(Xtrain.shape[1])

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    print()

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record3)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Epoch 1/40
107/107 [==============================] - 1s 5ms/step - loss: 0.6287 - accuracy: 0.6417 - val_loss: 0.5969 - val_accuracy: 0.6296
Epoch 2/40
107/107 [==============================] - 0s 2ms/step - loss: 0.5407 - accuracy: 0.7103 - val_loss: 0.5277 - val_accuracy: 0.7328
Epoch 3/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4891 - accuracy: 0.7616 - val_loss: 0.4848 - val_accuracy: 0.7487
Epoch 4/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4575 - accuracy: 0.7798 - val_loss: 0.4576 - val_accuracy: 0.7831
Epoch 5/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4407 - accuracy: 0.7913 - val_loss: 0.4475 - val_accuracy: 0.7804
Epoch 6/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4326 - accuracy: 0.8019 - val_loss: 0.4453 - val_accuracy: 0.7857
Epoch 7/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4184 - accuracy: 0.8048 - val_loss: 0.4396 - val_accuracy: 0.7910
Epoch 

107/107 [==============================] - 0s 3ms/step - loss: 0.3592 - accuracy: 0.8413 - val_loss: 0.4512 - val_accuracy: 0.7646
Epoch 16/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3577 - accuracy: 0.8381 - val_loss: 0.4540 - val_accuracy: 0.7540
Epoch 00016: early stopping
Test Accuracy: 77.24867463111877

Training 4: 
Epoch 1/40
107/107 [==============================] - 1s 5ms/step - loss: 0.6380 - accuracy: 0.6247 - val_loss: 0.5559 - val_accuracy: 0.6878
Epoch 2/40
107/107 [==============================] - 0s 3ms/step - loss: 0.5478 - accuracy: 0.7136 - val_loss: 0.4681 - val_accuracy: 0.7672
Epoch 3/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4923 - accuracy: 0.7645 - val_loss: 0.4396 - val_accuracy: 0.7989
Epoch 4/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4602 - accuracy: 0.7780 - val_loss: 0.4185 - val_accuracy: 0.7857
Epoch 5/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4506 - 

107/107 [==============================] - 0s 2ms/step - loss: 0.4244 - accuracy: 0.8034 - val_loss: 0.4594 - val_accuracy: 0.7825
Epoch 7/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4123 - accuracy: 0.8099 - val_loss: 0.4546 - val_accuracy: 0.7905
Epoch 8/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4136 - accuracy: 0.8093 - val_loss: 0.4553 - val_accuracy: 0.7798
Epoch 9/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3969 - accuracy: 0.8237 - val_loss: 0.4606 - val_accuracy: 0.7905
Epoch 10/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3894 - accuracy: 0.8246 - val_loss: 0.4845 - val_accuracy: 0.7878
Epoch 11/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3828 - accuracy: 0.8228 - val_loss: 0.4714 - val_accuracy: 0.8117
Epoch 12/40
107/107 [==============================] - 0s 3ms/step - loss: 0.3766 - accuracy: 0.8284 - val_loss: 0.4692 - val_accuracy: 0.7958
Epoch 13/40
10

107/107 [==============================] - 0s 3ms/step - loss: 0.5386 - accuracy: 0.7195 - val_loss: 0.4989 - val_accuracy: 0.7745
Epoch 3/40
107/107 [==============================] - 0s 2ms/step - loss: 0.4874 - accuracy: 0.7678 - val_loss: 0.4646 - val_accuracy: 0.7825
Epoch 4/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4578 - accuracy: 0.7878 - val_loss: 0.4741 - val_accuracy: 0.7666
Epoch 5/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4446 - accuracy: 0.7916 - val_loss: 0.4454 - val_accuracy: 0.7878
Epoch 6/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4358 - accuracy: 0.7978 - val_loss: 0.4439 - val_accuracy: 0.7984
Epoch 7/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4167 - accuracy: 0.8093 - val_loss: 0.4332 - val_accuracy: 0.8064
Epoch 8/40
107/107 [==============================] - 0s 3ms/step - loss: 0.4179 - accuracy: 0.8019 - val_loss: 0.4501 - val_accuracy: 0.7905
Epoch 9/40
107/10

## Summary

In [35]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,81.216931,78.306878,77.248675,82.804233,80.423278,81.43236,82.758623,81.167108,78.514588,80.636603,80.450928


In [36]:
report = record3
report = report.to_excel('Emb_MLP_CR_3.xlsx', sheet_name='model_3')